## Librerias

In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import json
# import plotly.express as px
# import seaborn as sbs

import warnings
warnings.filterwarnings("ignore")

## Parametros

In [2]:
client_bq = bigquery.Client()

In [12]:
main_table = 'co-keralty-costomedico.Diabetes_avicena_survival.diabetes'

query_for_t0_pop = f"""
SELECT
  numero_identificacion_paciente,
  year,
  month,
  DATE(CONCAT(year,'-',month,'-01')) AS fecha,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes,
FROM
  `{main_table}`
WHERE
  peso IS NOT NULL
  AND talla IS NOT NULL
  AND genero_paciente IS NOT NULL
  AND raza_paciente IS NOT NULL
  AND nivel_academico_paciente IS NOT NULL
  AND HDL IS NOT NULL
  AND LDL IS NOT NULL
  AND trigliceridos IS NOT NULL
  AND perimetro_abdominal IS NOT NULL
  AND hace_ejercicio IS NOT NULL
  AND diabetes = 0
"""

query_for_t1_pop = """

WITH
  no_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) DESC) AS orden
    FROM
      `co-keralty-costomedico.Diabetes_avicena_survival.diabetes`
    WHERE
      diabetes = 0
      AND numero_identificacion_paciente IN {})
  WHERE
    orden = 1),

si_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden
    FROM
      `co-keralty-costomedico.Diabetes_avicena_survival.diabetes`
    WHERE
      diabetes = 1
      AND numero_identificacion_paciente IN {})
  WHERE
    orden = 1)

    
SELECT
  *
FROM
  no_diabetes_t1
WHERE
  no_diabetes_t1.numero_identificacion_paciente NOT IN (
  SELECT
    DISTINCT numero_identificacion_paciente
  FROM
    si_diabetes_t1)
UNION ALL (
  SELECT
    *
  FROM
    si_diabetes_t1)

"""

## Funciones

In [4]:
def load_data(client, query, local = False, save = True, file_name = 'data'):

    if local:
        data = pd.read_csv(f'data/{file_name}.csv', sep = '|')
    else:
        data = client.query(query).result().to_dataframe()

    if save:
        data.to_csv(f'data/{file_name}.csv', sep = '|')

    return data

def etiquetar_diabetes(t0,t1):
    if t1 > t0:
        return t1
    else:
        return t0

## Proceso

In [5]:
fechas_minimas = client_bq.query('SELECT DISTINCT year, month FROM `co-keralty-costomedico.Diabetes_avicena_survival.diabetes` ORDER BY 1, 2 LIMIT 1').result().to_dataframe()
min_year = fechas_minimas['year'].values[0]
min_month = fechas_minimas['month'].values[0]

print(f'El tiempo inicial del estudio es {min_year}-{min_month}')

El tiempo inicial del estudio es 2022-1


In [13]:
# Carga de afiliados con datos en t0
data_t0 = load_data(client=client_bq, query=query_for_t0_pop, file_name='data_t0')
print(f'Se cuentan con {data_t0.shape[0]} afiliados con datos en el momento t0')
display(data_t0.head())

# Carga de afiliados que cumplen el evento (Diabetes)
afiliados_con_t0 = tuple(data_t0.numero_identificacion_paciente.unique())
data_t1 = load_data(client=client_bq, query=query_for_t1_pop.format(afiliados_con_t0,afiliados_con_t0), file_name='data_t1')
print(f'Se cuentan con {data_t1.shape[0]} afiliados que cumplen el evento (Diabetes) momento t1')
display(data_t1.head())

Se cuentan con 23083 afiliados con datos en el momento t0


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,43801872,2022,12,2022-12-01,55,68.500000000,1.490000000,30.854466015,40.50,141.00,155.0,110.000000000,Femenino,Mestizo,Básica secundaria,1,1,0,60 minutos\n,0
1,43283886,2022,12,2022-12-01,51,61.000000000,1.540000000,25.721032215,62.00,146.00,94.0,90.000000000,Femenino,Otros,Básica secundaria,1,1,0,Nunca\n,0
2,21243633,2022,12,2022-12-01,60,72.700000000,1.550000000,30.260145682,56.68,110.40,69.9,89.000000000,Femenino,Otros,Media académica o clásica,1,1,0,20 minutos\n,0
3,521644,2022,12,2022-12-01,95,66.000000000,1.680000000,23.384353741,55.90,143.00,122.0,93.000000000,Masculino,Otros,Básica secundaria,1,1,0,Nunca\n,0
4,19105608,2022,12,2022-12-01,72,65.800000000,1.615000000,25.227884864,63.00,42.96,106.7,84.000000000,Masculino,Otros,Básica secundaria,1,1,0,Nunca\n,0


Se cuentan con 23083 afiliados que cumplen el evento (Diabetes) momento t1


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,14960865,2022,12,2022-12-01,72,71.000000000,1.630000000,26.722872521,NaN,48.00,NaN,94.000000000,Masculino,Otros,Básica secundaria,1,1,0,None,1
1,6758605,2023,2,2023-02-01,68,82.000000000,1.700000000,28.373702422,36.0,70.41,105.2,98.000000000,Masculino,Otros,Básica secundaria,1,1,0,None,1
2,25050894,2023,2,2023-02-01,86,63.000000000,1.650000000,23.140495868,NaN,NaN,NaN,87.000000000,Femenino,Otros,Básica secundaria,1,1,0,None,1
3,40341203,2023,1,2023-01-01,38,77.966666667,1.620000000,29.708377788,NaN,NaN,NaN,92.500000000,Femenino,Otros,Ninguno,1,1,0,None,1
4,41342226,2023,2,2023-02-01,79,53.000000000,1.510000000,23.244594535,NaN,NaN,NaN,72.000000000,Femenino,Otros,Normalista,1,1,0,None,1


In [14]:
afiliados_with_no_t1 = list(data_t0[data_t0.fecha >= data_t1.fecha].numero_identificacion_paciente.unique())

data_t1_clean = data_t1[~data_t1.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t0_clean = data_t0[~data_t0.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t1_clean_to_merge = data_t1_clean[['numero_identificacion_paciente','diabetes']].rename(columns = {'diabetes':'diabetes_t1'})
full_data = data_t0_clean.merge(data_t1_clean_to_merge, on = 'numero_identificacion_paciente', how = 'left') #pd.concat([data_t0_clean,data_t1_clean])
full_data.diabetes = full_data.apply(lambda x: etiquetar_diabetes(x.diabetes, x.diabetes_t1), axis = 1)
full_data = full_data.drop(columns = 'diabetes_t1')

full_data

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,43283886,2022,12,2022-12-01,51,61.000000000,1.540000000,25.721032215,62.00,146.00,94.0,90.000000000,Femenino,Otros,Básica secundaria,1,1,0,Nunca\n,0
1,21243633,2022,12,2022-12-01,60,72.700000000,1.550000000,30.260145682,56.68,110.40,69.9,89.000000000,Femenino,Otros,Media académica o clásica,1,1,0,20 minutos\n,0
2,521644,2022,12,2022-12-01,95,66.000000000,1.680000000,23.384353741,55.90,143.00,122.0,93.000000000,Masculino,Otros,Básica secundaria,1,1,0,Nunca\n,0
3,19105608,2022,12,2022-12-01,72,65.800000000,1.615000000,25.227884864,63.00,42.96,106.7,84.000000000,Masculino,Otros,Básica secundaria,1,1,0,Nunca\n,0
4,31886481,2022,12,2022-12-01,60,60.000000000,1.520000000,25.969529086,47.60,104.70,116.0,85.000000000,Femenino,Otros,Profesional,1,1,0,Nunca\n,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,19406065,2023,4,2023-04-01,63,83.000000000,1.640000000,30.859607377,50.00,80.00,158.0,111.000000000,Masculino,Otros,Básica secundaria,1,1,0,Nunca\n,0
9800,21215421,2023,4,2023-04-01,83,65.700000000,1.580000000,26.317897773,53.00,219.00,155.0,96.000000000,Femenino,Otros,Normalista,1,0,0,Nunca\n,0
9801,21176918,2023,4,2023-04-01,57,69.000000000,1.570000000,27.993022029,45.00,201.60,234.0,95.000000000,Femenino,Otros,Bachillerato técnico,1,1,0,Nunca\n,0
9802,40012448,2023,4,2023-04-01,65,57.000000000,1.500000000,25.333333333,39.70,153.10,193.2,80.000000000,Femenino,Otros,Básica secundaria,1,1,0,20 minutos\n,0


In [54]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes


In [55]:
full_data[full_data.numero_identificacion_paciente == '79569003'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
5918,79569003,2023,9,2023-09-01,52,93.500000000,1.870000000,45.0,106.0,124.0,97.000000000,Masculino,Otros,Básica secundaria,1,0,0,Nunca\n,1


In [15]:
fechas = data_t1_clean[['numero_identificacion_paciente','year','month','fecha']].rename(columns = {'year':'year_t1','month':'month_t1'})

fechas['year_event'] = fechas.fecha.apply(lambda x: x.year)
fechas['month_event'] = fechas.fecha.apply(lambda x: x.month)
fechas.head(1)

,numero_identificacion_paciente,year_t1,month_t1,fecha,year_event,month_event
0,14960865,2022,12,2022-12-01,2022,12


In [16]:
times_to_event = data_t0_clean.sort_values(['numero_identificacion_paciente','year','month'])[['numero_identificacion_paciente','year','month','diabetes']]
times_to_event = times_to_event.merge(fechas, on = 'numero_identificacion_paciente', how = 'left')
times_to_event['time_to_event'] = abs((times_to_event.year - times_to_event.year_event)*12 + (times_to_event.month - times_to_event.month_event))
times_to_event = times_to_event[times_to_event.time_to_event != 0][['numero_identificacion_paciente','time_to_event']]

times_to_event

,numero_identificacion_paciente,time_to_event
0,1000048363,22
1,1000127649,10
3,1000322716,31
4,1000330502,20
5,1000464419,22
...,...,...
9797,9876080,14
9800,994635,8
9801,996466921011995,5
9802,9977679,29


In [17]:
full_data_with_time = full_data.merge(times_to_event, on = 'numero_identificacion_paciente', how = 'inner')
full_data_with_time[['peso','talla','perimetro_abdominal']] = full_data_with_time[['peso','talla','perimetro_abdominal']].astype(float)
full_data_with_time.hace_ejercicio = full_data_with_time.hace_ejercicio.apply(lambda x: x.replace('\n',''))

full_data_with_time

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
0,43283886,2022,12,2022-12-01,51,61.0,1.540,25.721032215,62.00,146.00,...,90.0,Femenino,Otros,Básica secundaria,1,1,0,Nunca,0,24
1,21243633,2022,12,2022-12-01,60,72.7,1.550,30.260145682,56.68,110.40,...,89.0,Femenino,Otros,Media académica o clásica,1,1,0,20 minutos,0,24
2,521644,2022,12,2022-12-01,95,66.0,1.680,23.384353741,55.90,143.00,...,93.0,Masculino,Otros,Básica secundaria,1,1,0,Nunca,0,19
3,19105608,2022,12,2022-12-01,72,65.8,1.615,25.227884864,63.00,42.96,...,84.0,Masculino,Otros,Básica secundaria,1,1,0,Nunca,0,24
4,31886481,2022,12,2022-12-01,60,60.0,1.520,25.969529086,47.60,104.70,...,85.0,Femenino,Otros,Profesional,1,1,0,Nunca,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9027,19406065,2023,4,2023-04-01,63,83.0,1.640,30.859607377,50.00,80.00,...,111.0,Masculino,Otros,Básica secundaria,1,1,0,Nunca,0,20
9028,21215421,2023,4,2023-04-01,83,65.7,1.580,26.317897773,53.00,219.00,...,96.0,Femenino,Otros,Normalista,1,0,0,Nunca,0,20
9029,21176918,2023,4,2023-04-01,57,69.0,1.570,27.993022029,45.00,201.60,...,95.0,Femenino,Otros,Bachillerato técnico,1,1,0,Nunca,0,19
9030,40012448,2023,4,2023-04-01,65,57.0,1.500,25.333333333,39.70,153.10,...,80.0,Femenino,Otros,Básica secundaria,1,1,0,20 minutos,0,19


In [18]:
pandas_gbq.to_gbq(full_data_with_time, 'Diabetes_avicena_survival.diabetes_final', project_id='co-keralty-costomedico',if_exists = 'replace')

In [60]:
full_data_with_time[full_data_with_time.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event


In [61]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
